In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

qualifying_schema =  StructType(fields=[
    StructField("qualifyId", IntegerType(), False),
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("constructorId", IntegerType(), True),
    StructField("number", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("q1",StringType(), True),
    StructField("q2", StringType(), True),
    StructField("q3", StringType(), True)
    ])

In [0]:
qualifying_df = spark.read.schema(qualifying_schema)\
    .option('multiLine', True)\
    .json(f'{raw_folder_path}/{v_file_date}/qualifying')

In [0]:
final_df = qualifying_df.withColumnRenamed('raceId', 'race_id')\
    .withColumnRenamed('driverId', 'driver_id')\
    .withColumnRenamed('constructorId', 'constructor_id')\
    .withColumnRenamed('qualifyId', 'qualify_id')\
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(v_file_date))


In [0]:
final_df = add_ingestion_date(final_df)

In [0]:

final_df.write.mode('overwrite').format('parquet').saveAsTable('f1_processed.qualifying')

In [0]:
dbutils.notebook.exit("Success")